## 路径还原

注：存在 86 个重复三元组，提交不知道会不会遇到这些情形
- 比如假设 triple1 即为 en 也为 zh，那么
- en -> zh -> en 和 zh -> zh -> en 都将作为答案

In [212]:
include("../src/CCKS-mKGQA.jl")
include("../src/loaddata/extractdata.jl") # 原始数据
include("../src/loaddata/translatedata.jl") # 翻译数据
include("../src/tools/ettalign.jl") # 对齐工具
include("../src/tools/distance.jl") # 距离函数
include("../src/process/submitpath.jl") # 数据提交
mkpath("submit")

"submit"

In [192]:
illpath = "ILLs_1"
# illpath = "ILLs_2"
# illpath = "ILLs_3"
# illpath = "ILLs_4"
# illpath = "ILLs_official"
# illpath = "ILLs_wiki"
include("../src/loaddata/illsdata.jl");

对齐数目-翻译后的数目：	[33801, 23998]


### 反转字典

In [117]:
# 元组逆向
println("重复元组的信息")
sort_count(length.(values(dict_triples_rev)))

重复元组的信息


OrderedDict{Int64, Int64} with 3 entries:
  1 => 260361
  2 => 3541
  3 => 14

### 精确匹配

In [182]:
submit_id = "precise-ill3model-ill4path"
sols = split.(strip.(split(strip(read(open("predict/predict_data_0708.txt", "r"), String)), '\n')),'\t')
noans = "<http://dbpedia.org/resource/Francis_Russell,_Marquess_of_Tavistock>#<http://dbpedia.org/property/parents>#<http://dbpedia.org/resource/John_Russell,_4th_Duke_of_Bedford>#<http://zh.dbpedia.org/resource/第四代贝德福德公爵约翰·罗素>#<http://zh.dbpedia.org/property/successor>#<http://zh.dbpedia.org/resource/丹尼尔·芬奇，第八代温奇尔西伯爵>";

# 返回结果
res = Vector{String}(undef, 1500)
fails, notgood = Int[], Int[]
## 精确匹配
for (i, sol) in enumerate(sols)
    que, rels = ill_valid_ques[i], sol[3:end]
    ner, rawner = ill_valid_ners[i], valid_ques_ner[i][2]
    # 精确路径
    submits = precise_submit(ner, rels, edges, dict_triples_rev)
    isempty(submits) && (push!(fails, i);continue)
    bettersubmits = filter(contains(rawner), submits)
    isempty(bettersubmits) ? push!(notgood, i) : (submits = bettersubmits)
    res[i] = first(submits)
end
println(length(fails), '\t', length(notgood))

## 剩下部分放弃
res[fails] .= noans
open("submit/submit_$submit_id.txt", "w") do io
    println(io, "id\tans_path")
    for (i, sol) in enumerate(res)
        println(io, i, '\t', sol)
    end
end

567	2


In [6]:
# ill3 + ill1
0.4673 / (1500 - 567) * 1500

0.7512861736334404

In [18]:
# ill3 + ill3
0.5247 / (1500 - 463) * 1500

0.7589681774349084

In [19]:
# ill3 + ill4
0.5453 / (1500 - 425) * 1500

0.7608837209302325

In [131]:
(0.5453 * 1500 + 183) /1500

0.6673

### 相似匹配

In [187]:
submit_id = "vague-ill3model-ill4path-ill1deduct"
sols = split.(strip.(split(strip(read(open("predict/predict_data_0708.txt", "r"), String)), '\n')),'\t')
noans = "<http://dbpedia.org/resource/Francis_Russell,_Marquess_of_Tavistock>#<http://dbpedia.org/property/parents>#<http://dbpedia.org/resource/John_Russell,_4th_Duke_of_Bedford>#<http://zh.dbpedia.org/resource/第四代贝德福德公爵约翰·罗素>#<http://zh.dbpedia.org/property/successor>#<http://zh.dbpedia.org/resource/丹尼尔·芬奇，第八代温奇尔西伯爵>";

# 返回结果
res = Vector{String}(undef, 1500)
fails, notgood = Int[], Int[]
## 精确匹配
for (i, sol) in enumerate(sols)
    que, rels = ill_valid_ques[i], sol[3:end]
    ner, rawner = ill_valid_ners[i], valid_ques_ner[i][2]
    # 精确路径
    submits = precise_submit(ner, rels, edges, dict_triples_rev)
    isempty(submits) && (push!(fails, i); res[i]=noans; continue)
    bettersubmits = filter(contains(rawner), submits)
    isempty(bettersubmits) ? push!(notgood, i) : (submits = bettersubmits)
    res[i] = first(submits)
end
println(length(fails), '\t', length(notgood))

425	8


In [193]:
# 训练集已有关系
dict_trainrels = DefaultDict{String, Vector{Tuple}}(Vector{Tuple})
for (que, _, rels...) in mt_train_ques_rels
    Tuple(rels) ∈ dict_trainrels[que] && continue
    push!(dict_trainrels[que], Tuple(rels))
end

In [214]:
## 剩下部分相似匹配 + 路径推理
failagain = Int[]
for i in fails
    rawner = ill_valid_ners[i]
    que = findbest(ill_train_ques, ill_valid_ques[i])
    submits = vcat([vague_submit(ner, rels, ill_triples, dict_triples_rev)
        for rels in dict_trainrels[que]]...)
    isempty(submits) && (push!(failagain, i); res[i]=noans; continue)
    bettersubmits = filter(contains(rawner), submits)
    isempty(bettersubmits) ? push!(notgood, i) : (submits = bettersubmits)
    res[i] = first(submits)
    print(i,'\t')
    flush(stdout)
end
length(failagain)

7	66	232	318	321	394	433	486	489	501	511	541	545	546	570	578	585	597	606	610	612	624	632	634	637	639	663	689	691	706	715	726	727	730	760	971	984	989	991	994	995	996	998	999	1133	1171	1186	1216	1218	1231	1232	1235	1245	1249	1264	1270	1282	1290	1294	1297	1303	1313	1315	1324	1331	1348	1350	1359	1361	1363	1391	1393	1396	1415	1452	1458	1459	1460	1465	1468	1487	1492	1495	

342

In [215]:
open("submit/submit_$submit_id.txt", "w") do io
    println(io, "id\tans_path")
    for (i, sol) in enumerate(res)
        println(io, i-1, '\t', sol)
    end
end

In [213]:
function dist(target, word; char=true)
    char || (words = equivills[word])
    target, word = lowercase.([target, word])
    char || (target = split(target, '_'); word = split(word, '_'))
    if char || isempty(words) # 问题匹配
        all(∉(Set(target)), word) && return typemax(Int)
        evaluate(Levenshtein(), target, word)
    else
        # 实体匹配
        words = split.(lowercase.(words), '_')
        all(all(∉(Set(target)), word) for word in words) && return typemax(Int)
        minimum(evaluate(Levenshtein(), target, word) for word in words)
    end
end

dist (generic function with 1 method)

In [204]:
equivills[""]

SubString{String}[]

In [64]:
64 / 463

0.13822894168466524

### 模糊匹配

In [54]:
submit_id = "pureill3"
sols = split.(strip.(split(strip(read(open("predict/predict_data_0708.txt", "r"), String)), '\n')),'\t')
noans = "<http://dbpedia.org/resource/Francis_Russell,_Marquess_of_Tavistock>#<http://dbpedia.org/property/parents>#<http://dbpedia.org/resource/John_Russell,_4th_Duke_of_Bedford>#<http://zh.dbpedia.org/resource/第四代贝德福德公爵约翰·罗素>#<http://zh.dbpedia.org/property/successor>#<http://zh.dbpedia.org/resource/丹尼尔·芬奇，第八代温奇尔西伯爵>";

fails = []
notgood = []
none = String[]
open("submit/submit_$(submit_id).txt", "w") do io
    println(io, "id\tans_path")
    for (i, sol) in enumerate(sols)
        que, rels = sol[1], sol[3:end]
        ner, rawner = ill_valid_ners[i], valid_ques_ner[i][2]
        res = precise_submit(ner, rels, edges, dict_triples_rev)
        if isempty(res) # 匹配不到答案
            push!(fails, [que, rels])
            for best in 1:8
                res = vague_submit(ner, rels, ill_triples, dict_triples_rev;best=best) # 寻找模糊路径
                isempty(res) || break
            end
            isempty(res) && (push!(none, que); res = [noans])
        else
            betterres = filter(str->occursin(rawner, str), res)
            isempty(betterres) ? push!(notgood, (i, rawner, ner, res)) : (res = betterres)
        end
        println(io, i-1, '\t', first(res))
    end
end
println("无解\t路径推理可解\t完全无解")
println([length(fails), length(fails)-length(none), length(none)])

无解	路径推理可解	完全无解
[463, 463, 0]


In [56]:
0.5673 * 1500 / (1500-463)

0.8205882352941177

In [28]:
open("nonsol0708.txt", "w") do io
    for (que, rels) in fails
        println(io, que, '\t', join(rels, '\t'))
    end
end

In [21]:
fails[1]

2-element Vector{Any}:
 "where is the constituency of th" ⋯ 26 bytes ⋯ ", a hong kong politician, from?"
 SubString{String}["alongside", "honorificsuffix"]

提交结果分析

 | ILLs | 准确率 | 完全无解 | 使用路径推理 | 缺二跳(验证集/图谱) | 缺三跳(验证集/图谱) | 
 | ---- | ----- | ----- | ----- | ------ | ---- |
 | 官方 | 0.3513 |573 | 323 | 156/52268 | 352/67437 | 
 | ILLs2 | | 551 | 221 | 98/47455 | 263/61439 | 
 | ILLs1 | 0.374 | 543 | 210 | 80/44396 | 209/57285 |
 | ILLs3 | 0.3833 | 535 | 183 | 61/37283 | 137/48256 |
 | wiki | 0.422 | 478 | 134 | 6/38070 | 103/49921 |
 | ill3 模型 + wiki 路径| 0.692 | 175 | 12 | - | - |
 | ill3 模型 + ill3 路径| 0.5673 | 301 | 162 | - | - |
 | ill3 模型 + ill4 路径| 0.5753 | 260 | 165 | - | - |
 | wiki 模型 + ill3 路径| 0.612 | - | - |
 | wiki 模型 + ill4 路径| 0.6193 | 190 | 158 | - | - |
 | wiki 模型 + wiki 路径| 0.734 | - | - | - | - |
 | ill4 模型 + ill4 路径| 0.5993 | - | - | - | - |


In [48]:
0.5673 * 1500 / (1500-462)

0.8197976878612717

### 测试对齐

In [7]:
include("../src/regexpr.jl")
function get_rel(link)
    zh = get_txt(zh_rel_reg, link)
    dict_rels[isempty(zh) ? get_txt(en_rel_reg, link) : zh]
end

get_rel (generic function with 1 method)

In [8]:
sols = last.(split.(split(strip(read(open("submit_11.txt", "r"), String)), '\n')[2:end], '\t'))
sols = [sol[2:3:end] for sol in split.(sols, '#')]
sols = [get_rel.(sol) for sol in sols];

In [61]:
submit_id = 21
noans = "<http://dbpedia.org/resource/Francis_Russell,_Marquess_of_Tavistock>#<http://dbpedia.org/property/parents>#<http://dbpedia.org/resource/John_Russell,_4th_Duke_of_Bedford>#<http://zh.dbpedia.org/resource/第四代贝德福德公爵约翰·罗素>#<http://zh.dbpedia.org/property/successor>#<http://zh.dbpedia.org/resource/丹尼尔·芬奇，第八代温奇尔西伯爵>";

fails = String[]
notgood = []
none = String[]
open("submit/submit_$(submit_id).txt", "w") do io
    println(io, "id\tans_path")
    for (i, rels) in enumerate(sols)
        que = mt_valid_ques[i]
        ner, rawner = ill_valid_ners[i], valid_ques_ner[i][2]
        res = precise_submit(ner, rels, edges, dict_triples_rev)
        if isempty(res) # 匹配不到答案
            push!(fails, que)
            res = vague_submit(ner, rels, ill_triples, dict_triples_rev) # 寻找模糊路径
            isempty(res) && (push!(none, que); res = [noans])
        else
            betterres = filter(str->occursin(rawner, str), res)
            isempty(betterres) ? push!(notgood, (i, rawner, ner, res)) : (res = betterres)
        end
        println(io, i-1, '\t', first(res))
    end
end
length(fails), length(notgood), length(none)

(718, 0, 535)

### 对齐程度分析

In [ ]:
# ILLs 3
# 535 完全无解 183 路径推理
# 训练集
# 缺二跳 37283 缺三跳 48256
# 验证集
# 缺二跳 61 缺三跳 137

In [16]:
# wiki ILLs
# 478 完全无解 134 路径推理
# 训练集
# 缺二跳 38070 缺三跳 49921
# 验证集
# 缺二跳 6 缺三跳 103

### 关系抽取

In [106]:
for que in mt_valid_ques
    nears = top10(que)
end

1500-element Vector{String}:
 "where is the constituency of th" ⋯ 26 bytes ⋯ ", a hong kong politician, from?"
 "who is the successor of the parent of NER, an Irish politician?"
 "who is the leader of the administrative region to which NER belongs?"
 "who preceded the parent of NER, an american economist?"
 "which draft team does the autho" ⋯ 60 bytes ⋯ "ariq and peter gunz, belong to?"
 "who is before the parent of NER" ⋯ 54 bytes ⋯ " minister of india (1889-1964)?"
 "where does the location province of NER, a mine in mongolia, belong to?"
 "who does the origin of NER influence's name?"
 "what is the genre of the author of NER, a song performed by carlos santana?"
 "which title does the parent of NER, an american economist, belong to?"
 "what is the parent mountain of the client of NER?"
 "who did a job before the parent of NER?"
 "what is the type of the people, things and things with the same name as NER?"
 ⋮
 "do you know what time zone is i" ⋯ 66 bytes ⋯ " community of catalonia, 

In [247]:
rel_rules = (
    r"(where|what|who|which|do you know) " => " ",
    r" (does|did|does|the|that|is|of) " => " ",
    r"^(does|did|does|the|that|is) " => " ",
    r", (a|an) .*?[,?]" => "",
    r" {2,}" => " ",
    # "belong to" => "",
    "?" => "",
    "NER's" => "",
    "NER" => ""
)
template_train_ques = String[]
for que in mt_train_ques
    que = strip(replace(que, rel_rules...))
    que = strip(replace(que, rel_rules...))
    que = strip(replace(que, rel_rules...))
    push!(template_train_ques, que)
end

template_valid_ques = String[]
for que in mt_valid_ques
    que = strip(replace(que, rel_rules...))
    que = strip(replace(que, rel_rules...))
    que = strip(replace(que, rel_rules...))
    push!(template_valid_ques, que)
end

# open("template/valid_ques.txt", "w") do io
#     # for que in mt_valid_ques
    
# end

In [248]:
filter(∈(Set(template_train_ques), template_valid_ques)

650-element Vector{String}:
 "constituency one alongside from"
 "successor parent"
 "location province belong to"
 "origin influence's name"
 "genre author"
 "title parent belong to"
 "parent mountain client"
 "a job before parent"
 "profession one after election"
 "before commander"
 "a job before one after election"
 "spoken by name included in"
 "married author"
 ⋮
 "title The winner"
 "level management at Southeast Site"
 "Model Regulation The province located"
 "time zone in East location"
 "nationality Election after"
 "eponymous Alma Mater"
 "administrative center Western County"
 "title Election after"
 "was Election after before"
 "time zone in South spain"
 "are eponymous affected people"
 "predecessors The leader"

In [224]:
txts = []
for (i, (que, _, rels...)) in enumerate(mt_train_ques_rels)
    newque = strip(replace(que, rel_rules...))
    newque = strip(replace(newque, rel_rules...))
    newque = strip(replace(newque, rel_rules...))
    occursin("where", newque) && (push!(txts, join([newque, '|', rels...], '\t')))
end
open("template/where.txt", "w") do io
    for txt in unique(txts)
        println(io, txt)
    end
end

In [ ]:
txt

In [240]:
mt_train_ques_rels[[93, 223,227, 381]]

4-element Vector{Vector{SubString{String}}}:
 ["what is the leader of NER?", "Galactic_Empire_(Star_Wars)", "leader", "occupation"]
 ["what is the eponym of NER?", "Drygalski_(crater)", "eponym", "occupation"]
 ["what is the eponym of NER?", "Agatharchides_(crater)", "eponym", "occupation"]
 ["what is the singer of NER?", "I'm_Talking_with_My_Mouth", "extra", "occupation"]

In [244]:
mysols = []
for (i, que) in enumerate(template_ques)
    word = split(que)
    length(word) == 1 && println(join(mt_train_ques_rels[i][[1,3:end...]], '\t'))
    # length(word) == 1 && push!(mysols, [i, word])
    # length(word) == 2 && push!(mysols, [i, word])
end
mysols

what is the leader of NER?	leader	occupation
what is the eponym of NER?	eponym	occupation
what is the eponym of NER?	eponym	occupation
what is the singer of NER?	extra	occupation
what is the singer of NER, an album by alexz johnson?	extra	occupation
what is the singer of NER, a 2014 animated film by phil lord and chris miller?	extra	profession
what is the parent of NER, a hong kong banker and politician?	parents	occupation
what is the parent of NER, a hong kong businessman?	parents	occupation
what is the parent of NER, an american businessman and diplomat (1914-1993)?	parents	occupation
what is the author of NER?	chronology	profession
what is the author of NER?	chronology	profession
what is the author of NER, an album by vixx?	chronology	profession
what is the author of NER?	chronology	profession
what is the client of NER, a villa?	client	occupation
what is the author of NER, a live album?	chronology	occupation
what is the author of NER?	chronology	occupation
what is the author of NER,

Any[]

In [231]:
mt_train_ques_rels[[5808, 7670, 7929, 12689]]

4-element Vector{Vector{SubString{String}}}:
 ["where is the NER incident", "Weimar_Republic", "event", "location"]
 ["where is NER's team?", "Holsten-Deutschland-Cup", "team", "ground"]
 ["who is the producer of the creator of Danny Elfman's compilation album NER?", "Nightmare_Revisited", "chronology", "producer"]
 ["do you know where is the NER name", "List_of_administrative_divisions_of_Ili", "name", "location"]

In [125]:
match(r"(\w*) of (\w*)", template_ques[1])

RegexMatch("constituency of one", 1="constituency", 2="one")

In [139]:
filter(!isnothing, match.(r"^(.*)of(.*)$", template_ques))

1137-element Vector{Union{Nothing, RegexMatch}}:
 RegexMatch("constituency of one alongside", 1="constituency ", 2=" one alongside")
 RegexMatch("successor of parent of", 1="successor of parent ", 2="")
 RegexMatch("leader of administrative region to belongs", 1="leader ", 2=" administrative region to belongs")
 RegexMatch("preceded parent of", 1="preceded parent ", 2="")
 RegexMatch("draft team author of", 1="draft team author ", 2="")
 RegexMatch("before parent of", 1="before parent ", 2="")
 RegexMatch("location province of", 1="location province ", 2="")
 RegexMatch("origin of influence's name", 1="origin ", 2=" influence's name")
 RegexMatch("genre of author of", 1="genre of author ", 2="")
 RegexMatch("title parent of", 1="title parent ", 2="")
 RegexMatch("parent mountain of client of", 1="parent mountain of client ", 2="")
 RegexMatch("a job before parent of", 1="a job before parent ", 2="")
 RegexMatch("type of people, things and things with same name as", 1="type ", 2=" peopl

In [138]:
filter(!isnothing, match.(r"^(.*)of(.*)of(.*)$", template_ques))

485-element Vector{Union{Nothing, RegexMatch}}:
 RegexMatch("successor of parent of", 1="successor ", 2=" parent ", 3="")
 RegexMatch("genre of author of", 1="genre ", 2=" author ", 3="")
 RegexMatch("parent mountain of client of", 1="parent mountain ", 2=" client ", 3="")
 RegexMatch("profession of one after election of", 1="profession ", 2=" one after election ", 3="")
 RegexMatch("one known as prime minister of one after election of", 1="one known as prime minister ", 2=" one after election ", 3="")
 RegexMatch("headquarters of combatant of", 1="headquarters ", 2=" combatant ", 3="")
 RegexMatch("title of eponym of", 1="title ", 2=" eponym ", 3="")
 RegexMatch("leader of combatant of", 1="leader ", 2=" combatant ", 3="")
 RegexMatch("house of commander of", 1="house ", 2=" commander ", 3="")
 RegexMatch("successor of leader of", 1="successor ", 2=" leader ", 3="")
 RegexMatch("location of alma mater of", 1="location ", 2=" alma mater ", 3="")
 RegexMatch("succession of commander of"

In [136]:
filter(!isnothing, match.(r"^(\w*) of (\w*) of (\w*) of$", template_ques))

11-element Vector{Union{Nothing, RegexMatch}}:
 RegexMatch("legislature of country of birth of", 1="legislature", 2="country", 3="birth")
 RegexMatch("source of information of combatant of", 1="source", 2="information", 3="combatant")
 RegexMatch("timezong of east of south of", 1="timezong", 2="east", 3="south")
 RegexMatch("timezong of east of south of", 1="timezong", 2="east", 3="south")
 RegexMatch("deathpalce of successor of descendant of", 1="deathpalce", 2="successor", 3="descendant")
 RegexMatch("timezong of west of location of", 1="timezong", 2="west", 3="location")
 RegexMatch("title of consort of ruler of", 1="title", 2="consort", 3="ruler")
 RegexMatch("successor of successor of descendants of", 1="successor", 2="successor", 3="descendants")
 RegexMatch("successor of successor of descendants of", 1="successor", 2="successor", 3="descendants")
 RegexMatch("form of title of venue of", 1="form", 2="title", 3="venue")
 RegexMatch("father of ruler of descendants of", 1="father", 

In [95]:
"匹配最近 10 个问题"
templates = unique(mt_train_ques)
top10 = nearby(templates)

(::Base.Fix1{typeof(nearby), Vector{String}}) (generic function with 1 method)